This notebook shows how to do certain SQL commands in SQL Alchemy Core. It is designed as a reference for my own memory, and no other purpose.

In [15]:
# Imports
import pandas as pd
from sqlalchemy import create_engine, Table, MetaData, select, and_, or_

In [34]:
# I like to put everything into a dataframe. This funciton is just here to simplify the
# the queries and hide these details.
def get_results_frame(stmt):
    results = connection.execute(stmt).fetchall()
    df = pd.DataFrame(results)
    df.columns = results[0].keys()
    return df

In [35]:
# Create and engine object to access the database.
engine = create_engine('sqlite:///Chinook_Sqlite.sqlite')
connection = engine.connect()

In [36]:
# How to view all the tables in the database.
m = MetaData()
m.reflect(engine)
for table in m.tables.values():
    print(table.name)

Album
Artist
Customer
Employee
Genre
Invoice
InvoiceLine
Track
MediaType
Playlist
PlaylistTrack


In [37]:
# The following will reflect all of the tables in the database and store
# the reflections in a dictionary. This is probably not very smart for large databases.
metadata = m
tables = None
tables = {}
for table in m.tables.values():
    tables[table.name] = Table(table.name, metadata, autoload=True, autoload_with=engine)

In [38]:
# Showing all the tables again as keys in our dictionary.
tables.keys()

dict_keys(['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'Track', 'MediaType', 'Playlist', 'PlaylistTrack'])

In [39]:
# Show columns in a table
album = tables['Album']
print(album.c.keys())

['AlbumId', 'Title', 'ArtistId']


In [41]:
# A simple select statement.
album = tables['Album']
stmt = select([album.c.Title])
df = get_results_frame(stmt)
print(stmt)
df.head()

SELECT "Album"."Title" 
FROM "Album"


,Title
0,For Those About To Rock We Salute You
1,Balls to the Wall
2,Restless and Wild
3,Let There Be Rock
4,Big Ones


In [42]:
# A simple select with a where clase
stmt = select([album.c.Title])
stmt = stmt.where(album.c.Title == 'For Those About To Rock We Salute You')
df = get_results_frame(stmt)
print(stmt)
df.head()

SELECT "Album"."Title" 
FROM "Album" 
WHERE "Album"."Title" = :Title_1


,Title
0,For Those About To Rock We Salute You


In [44]:
# A simple select with a compound where clause using or_
stmt = select([album.c.Title])
stmt = stmt.where(or_(album.c.Title == 'For Those About To Rock We Salute You',
                      album.c.Title == 'Restless and Wild'))
df = get_results_frame(stmt)
print(stmt)
df.head()

SELECT "Album"."Title" 
FROM "Album" 
WHERE "Album"."Title" = :Title_1 OR "Album"."Title" = :Title_2


,Title
0,For Those About To Rock We Salute You
1,Restless and Wild
